<a href="https://colab.research.google.com/github/lifewinning/MLTSA22_final/blob/main/MLSTA_Final_Project_Preprocessing_with_Timestamps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Updated method for collecting imagery and incorporating timestamps.


## Initialize EarthEngine, import geemap



In [ ]:
pip install geemap

     |████████████████████████████████| 2.0 MB 8.5 MB/s 
     |████████████████████████████████| 3.3 MB 39.1 MB/s 
     |████████████████████████████████| 8.7 MB 35.3 MB/s 
     |████████████████████████████████| 1.2 MB 42.7 MB/s 
     |████████████████████████████████| 187 kB 62.3 MB/s 
     |████████████████████████████████| 95 kB 4.1 MB/s 
     |████████████████████████████████| 98 kB 8.2 MB/s 
     |████████████████████████████████| 99 kB 9.4 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
     |████████████████████████████████| 1.3 MB 45.5 MB/s 
     |████████████████████████████████| 130 kB 56.8 MB/s 
     |████████████████████████████████| 3.0 MB 39.7 MB/s 
     |████████████████████████████████| 128 kB 23.0 MB/s 
     |████████████████████████████████| 428 kB 68.5 MB/s 
     |████████████████████████████████| 342 kB 72.2 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 79 kB 5.7 MB/s 
     |█████████████████████

In [ ]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=5F3i3vkqWqvdO34shNI8Da5avNpTYKbLTdHb8jUVriQ&tc=YkS1vX8zLDDcfWVLfjGeml1OZOMEaIMWJwYvfp95LXk&cc=Gadv2-p7NmrstnP6PD-8fCmQIKKSu9Tuu5Dll2wEVG0

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgZqjQSpDatQO48BOVaN7ZJC7HVEfGr_fcVggtV6rEqHjtH2-B3L70

Successfully saved authorization token.


In [ ]:
import geemap
Map = geemap.Map()

In [ ]:
import json
from urllib.request import urlopen
import glob
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/MLTSA22

/content/drive/MyDrive/MLTSA22


# Set date params and load clipping features

In [ ]:
# establish time series window
start = ee.Date('2018-01-01')
finish = ee.Date('2021-12-31')

In [ ]:
# load geojson with bounding boxes
url = urlopen('https://raw.githubusercontent.com/lifewinning/MLTSA22_final/main/mltsa_bb.geojson')
original_sites = json.load(url)

In [ ]:
# there are 2 files bc I'm an idiot
url = urlopen('https://raw.githubusercontent.com/lifewinning/MLTSA22_final/main/added_ports.geojson')
geojson = json.load(url)

feats = geojson['features'] + original_sites['features']

for site, index in enumerate(range(0,len(feats))):
  print(index, feats[site]['properties']['site'])

0 oakland
1 tacoma
2 shanghai
3 bayonne
4 bayonne-02
5 savannah
6 miami
7 singapore
8 suez
9 long beach
10 rotterdam


# Some generalizing functions

In [ ]:
def cliptoaoi(image):
  return image.clip(aoi)

In [ ]:
# generate a range of days between start and end of time series
def getdates(collection, days):
  first = collection.first().date()
  diff = finish.difference(first, 'day')
  
  def getDays(day):
    return first.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1), days).map(getDays)

In [ ]:
def day_mosaics(date, newlist):
  date = ee.Date(date)
  newlist = ee.List(newlist)
  
  filtered = collection.filterDate(date, date.advance(1, 'day'))

  date_formatted = date.format('yyyy-MM-dd:H:mm:ss', 'UTC')

  dates = filtered.aggregate_array('system:time_start').map(lambda time_start: ee.Date(time_start).format('Y-MM-dd_H:mm:ss', 'UTC'))

  image_m = ee.Image(filtered.mosaic()).set({'ID': date_formatted}).set({'dates': dates}).clip(aoi)

  # Add the mosaic to a list only if the collection has images
  return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image_m), newlist))

In [ ]:
def check_gaps(list):
  dates = list.aggregate_array('system:time_start').getInfo()
  date1 = ee.Date(dates[0])
  date2 = ee.Date(dates[1])
  if date2.difference(date1, 'days').getInfo() > 6: 
    return 12
  if date2.difference(date1, 'days').getInfo() < 6: 
    return 6

In [ ]:
def clip(coords):
  return ee.Geometry.Polygon(coords['geometry']['coordinates'])

In [ ]:
def view_thumbnail(coords, band, number):
  name = coords['properties']['site']
  # query EE

  collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')

  ids = collection_clipped.aggregate_array('system:time_start').map(lambda time_start: ee.Date(time_start).format('Y-MM-dd_H:m:s', 'UTC')).getInfo()
 
  # take a quick look at the first image just to see

  print(name)
  out_img = "tests/{0}/1/{2}.png".format(band, number, name)

  vis_params = {'min': -5, 'max': 0, 'bands':'VH'}
  
  image = 'test/VH/5/{0}'.format(name)

  image = collection_clipped.first()
  geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
  geemap.show_image(out_img)

In [ ]:
def get_images(coords, band, number):
  name = coords['properties']['site']
  # query EE

  collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')

  ids = collection_clipped.aggregate_array('system:time_start').map(lambda time_start: ee.Date(time_start).format('Y-MM-dd_H:m:s', 'UTC')).getInfo()
  # take a quick look at the first image just to see

  print(name)
  newcol = ee.ImageCollection(ee.List(getdates(collection_clipped, check_gaps(collection_clipped)).iterate(day_mosaics, ee.List([]))))
  newcol_list = newcol.toList(newcol.size())
  vis_params = {'min': number, 'max': 0, 'bands': band }
  
  out_dir = 'timestampedit/{0}/{1}/{2}'.format(band, number, name)
  geemap.get_image_collection_thumbnails(collection_clipped, out_dir, vis_params, dimensions=800, format="png", names=ids)

In [ ]:
for f in feats: 
  aoi = clip(f)
  view_thumbnail(f, 'VH', -5)

oakland


Output()

tacoma


Output()

shanghai


...

Output()

long beach


Output()

rotterdam


Output()

In [ ]:
for f in feats: 
  aoi = clip(f)
  view_thumbnail(f, 'VH', -20)

oakland


Output()

tacoma


Output()

shanghai


...

Output()

long beach


Output()

rotterdam


Output()

In [ ]:
# export all images (uneven time series/large areas aren't tiled together)
# for f in feats: 
#   aoi = clip(f)
#   get_images(f, 'VH', -20)

oakland


NameError: ignored

Mosaic images (composed of images taken within the same day tiled together) are a little more finicky...

# Suez

In [ ]:
aoi = clip(feats[8])
name = feats[8]['properties']['site']
# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')

# mosaic images from the same day as needed
newcol = ee.ImageCollection(ee.List(getdates(collection_clipped, 2).iterate(day_mosaics, ee.List([]))))
dates = newcol.aggregate_array('dates').getInfo()
ids = []


for d in dates:
  if len(d) > 1: 
    to_dates = [datetime.strptime(item, '%Y-%m-%d_%H:%M:%S') for item in d]
    to_dates.index(min(to_dates))
    ids.append(d[to_dates.index(max(to_dates))])
  else:
    ids.append(d[0])

print(ids)

vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

out_dir = 'mosaic_with_timestamps/{0}'.format(name)
geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=800, format="png", names=ids)

['2018-01-04_15:48:05', '2018-01-06_3:43:43', '2018-01-10_15:47:39', '2018-01-12_3:44:31', '2018-01-16_15:48:05', '2018-01-18_3:43:43', '2018-01-22_15:47:39', '2018-01-24_3:44:31', '2018-01-28_15:48:04', '2018-01-30_3:43:42', '2018-02-03_15:47:38', '2018-02-05_3:44:31', '2018-02-09_15:48:04', '2018-02-11_3:43:42', '2018-02-15_15:47:38', '2018-02-17_3:44:30', '2018-02-21_15:48:04', '2018-02-23_3:43:42', '2018-02-27_15:47:38', '2018-03-01_3:44:30', '2018-03-05_15:48:04', '2018-03-07_3:43:42', '2018-03-11_15:47:38', '2018-03-13_3:44:30', '2018-03-17_15:48:04', '2018-03-19_3:43:42', '2018-03-23_15:47:38', '2018-03-25_3:44:31', '2018-03-29_15:48:04', '2018-03-31_3:43:42', '2018-04-04_15:47:39', '2018-04-06_3:44:31', '2018-04-10_15:48:05', '2018-04-12_3:43:43', '2018-04-16_15:47:39', '2018-04-18_3:44:31', '2018-04-22_15:48:05', '2018-04-24_3:43:43', '2018-04-28_15:47:40', '2018-04-30_3:44:32', '2018-05-04_15:48:06', '2018-05-06_3:43:44', '2018-05-10_15:47:40', '2018-05-12_3:44:33', '2018-05-

# Rotterdam

In [ ]:
aoi = clip(feats[10])
name = feats[10]['properties']['site']
# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection.size())

# mosaic images from the same day as needed
newcol = ee.ImageCollection(ee.List(getdates(collection_clipped, 2).iterate(day_mosaics, ee.List([]))))
ids = newcol.aggregate_array('dates')..getInfo()

print(ids)
# this is for exporting
# vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

# out_dir = 'mosaic/2/clip/{0}'.format(name)
# geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=500, format="png", names=ids)

[['2018-01-01_5:58:12'], ['2018-01-13_5:58:12'], ['2018-01-25_5:58:12'], ['2018-04-07_5:58:12'], ['2018-04-19_5:58:12'], ['2018-05-01_5:58:13'], ['2018-05-13_5:58:13'], ['2018-05-25_5:58:14'], ['2018-06-06_5:58:15'], ['2018-06-18_5:58:16'], ['2018-06-30_5:58:16'], ['2018-07-12_5:58:17'], ['2018-07-24_5:58:18'], ['2018-08-05_5:58:18'], ['2018-08-17_5:58:19'], ['2018-09-10_5:58:20'], ['2018-09-22_5:58:21'], ['2018-10-04_5:58:21'], ['2018-10-16_5:58:21'], ['2018-10-28_5:58:21'], ['2018-11-09_5:58:21'], ['2018-11-21_5:58:21'], ['2018-12-03_5:58:20'], ['2018-12-15_5:58:19'], ['2018-12-27_5:58:19'], ['2019-01-08_5:58:19'], ['2019-01-20_5:58:18'], ['2019-02-13_5:58:18'], ['2019-02-25_5:58:18'], ['2019-03-09_5:58:18'], ['2019-03-21_5:58:18'], ['2019-04-02_5:58:18'], ['2019-04-14_5:58:18'], ['2019-04-26_5:58:19'], ['2019-05-08_5:58:19'], ['2019-05-20_5:58:20'], ['2019-06-01_5:58:20'], ['2019-06-13_5:58:21'], ['2019-06-25_5:58:22'], ['2019-07-07_5:58:23'], ['2019-07-19_5:58:23'], ['2019-07-31_5: